# Finetuning Google Gemma's Model with CSV [data](https://drive.google.com/file/d/1Kkllt9AFmIz1K2Oe7AJpNmXTxoODIwht/view?usp=sharing)

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0
!pip3 install -q -U huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

In [ ]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [ ]:
from getpass import getpass
os.environ["HF_TOKEN"] = getpass("Your Huggingface Hub Token: ")

In [ ]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [ ]:
def formatting_func(example):
    text = f"Question: {example['inst'][0]}\n Answer: {example['answer'][0]}"
    return [text]

In [ ]:
from datasets import load_dataset
data = load_dataset('csv', data_files={'train': 'file.csv'})
data = data.map(lambda samples: tokenizer(samples["inst"]), batched=True)

In [ ]:
data['train']

Dataset({
    features: ['inst', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 11
})

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100, # should be changed if data is too less
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/11 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
1,0.543900
2,0.543900
3,0.534400
4,0.514100
5,0.493900
6,0.474000
7,0.453500
8,0.431300
9,0.406600
10,0.379700


TrainOutput(global_step=100, training_loss=0.0771272501032945, metrics={'train_runtime': 71.8958, 'train_samples_per_second': 5.564, 'train_steps_per_second': 1.391, 'total_flos': 595117043712000.0, 'train_loss': 0.0771272501032945, 'epoch': 100.0})

In [ ]:
text = "Question: The Car Doctors Ltd. 547 High Street, London Co. Reg. No.: 12345678 VAT No.GB123456789 Email: contact@thecardoctors.co.uk Phone: 07831261234 Website: thecardoctors.co.uk Bill to: John Smith 123 Piccadilly, London +447812345678 United Kingdom Invoice: Invoice Date: Due Date: 2021-009 19/05/2021 02/06/2021 Description Dent repair 6-inch dent above back left tyre Quantity 1 Unit Price VAT Amount each 200.00 20% 200.00 New tyre Standard multi-directional 1 each 60.00 20% 60.00 Tyre replacement Back left tyre - standard multi-directional 1 each 20.00 20% 20.00 Subtotal without VAT VAT 20% of 280.00 Total GBP Amount Paid Amount Due (GBP) 280.00 56.00 336.00 0.00 336.00 Terms & Conditions Vehicle: Black Ford Fiesta - AB12 CDE Payment to be made within 14 days via the payment link below. HH Pay online Bank Barclays Sort Code: 12-34-56 Account No.: 7654321 Account Holder: The Car Doctors Ltd. SWIFT: ABCDEF11 IBAN: GB11 HBUK 1111 1111 1111 11 Go to: https://invoice.sumup.com/s/OEsrHH Dm Or scan the QR code using your phone's camera. & sumup• Page 1 of 1 for Invoice #2021-009"

In [ ]:
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Question: The Car Doctors Ltd. 547 High Street, London Co. Reg. No.: 12345678 VAT No.GB123456789 Email: contact@thecardoctors.co.uk Phone: 07831261234 Website: thecardoctors.co.uk Bill to: John Smith 123 Piccadilly, London +447812345678 United Kingdom Invoice: Invoice Date: Due Date: 2021-009 19/05/2021 02/06/2021 Description Dent repair 6-inch dent above back left tyre Quantity 1 Unit Price VAT Amount each 200.00 20% 200.00 New tyre Standard multi-directional 1 each 60.00 20% 60.00 Tyre replacement Back left tyre - standard multi-directional 1 each 20.00 20% 20.00 Subtotal without VAT VAT 20% of 280.00 Total GBP Amount Paid Amount Due (GBP) 280.00 56.00 336.00 0.00 336.00 Terms & Conditions Vehicle: Black Ford Fiesta - AB12 CDE Payment to be made within 14 days via the payment link below. HH Pay online Bank Barclays Sort Code: 12-34-56 Account No.: 7654321 Account Holder: The Car Doctors Ltd. SWIFT: ABCDEF11 IBAN: GB11 HBUK 1111 1111 1111 11 Go to: https://invoice.sumup.com/s/OEsrHH D

In [ ]:
# Save the model and tokenizer locally
model.save_pretrained("gemma-arya")
tokenizer.save_pretrained("gemma-arya")

('gemma-arya/tokenizer_config.json',
 'gemma-arya/special_tokens_map.json',
 'gemma-arya/tokenizer.model',
 'gemma-arya/added_tokens.json',
 'gemma-arya/tokenizer.json')

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoConfig
from huggingface_hub import HfApi

In [ ]:
# Define your model repository name
repo_name = "/content/gemma-arya"

# Push to Hugging Face Hub
api = HfApi()
api.create_repo(repo_id="aryatito/my-gemma-model-csv", exist_ok=True)
api.upload_folder(
    repo_id="aryatito/my-gemma-model-csv",
    folder_path=repo_name,
    repo_type="model",
    commit_message="Initial commit",
    use_auth_token=True
)

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aryatito/my-gemma-model-csv/commit/64ddf56f588e82398fcc4cc75480507b554bf52e', commit_message='Initial commit', commit_description='', oid='64ddf56f588e82398fcc4cc75480507b554bf52e', pr_url=None, pr_revision=None, pr_num=None)